In [25]:
using Pkg
Pkg.activate(".") # Activate the environment
using ArgParse
using Logging, LoggingExtras
using JSON
using FilePathsBase
using DataStructures
using PropDicts
using Tables
using TypedTables
using LegendDataManagement
using Random, LinearAlgebra, Statistics, Distributions, StatsBase
using BAT, DensityInterface, IntervalSets
using TypedTables
using Plots
using Cuba

  Activating project at `~/LEGEND/ZeroNuFit.jl`


In [18]:
function build_prior(partitions;config,stat_only=false)
"""
Builds the priors for use in the fit
Parameters
    - partitions:Table of the partition info
    - config: the Dict of the fit config
    - stat_only; a bool for whether systematic uncertatinties are considered on energy scale
"""
    res=[]
    bias=[]
    for part in partitions
        append!(res,[Normal(part.fwhm/2.355,part.fwhm_sigma/2.355)])
        append!(bias,[Normal(part.bias,part.bias_sigma)])
    end
    if (stat_only==false)
        return distprod(S=0..config["upper_signal"],B=0..config["upper_bkg"], res= fill(0..10, length(partitions)),
    bias= fill(-2..2,length(partitions)))
    else 
        distprod(S=0..config["upper_signal"],B=0..config["upper_bkg"])
    end
    


end

build_prior (generic function with 1 method)

In [22]:
mu = [1,2,3,4,5,6,7,8,9,10]
sigma=ones(10)
print(mu,sigma)
res_list = Vector{Normal{Float64}}(undef,length(mu))

for (i,(m,s)) in enumerate(zip(mu,sigma))
    res_list[i]=Normal(m,s)
end

print("\n")
println(res_list)
distprod(S=0..1,B=0..1, res= fill(0..10, length(10)),
    bias= res_list)


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10][1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Normal{Float64}[Normal{Float64}(μ=1.0, σ=1.0), Normal{Float64}(μ=2.0, σ=1.0), Normal{Float64}(μ=3.0, σ=1.0), Normal{Float64}(μ=4.0, σ=1.0), Normal{Float64}(μ=5.0, σ=1.0), Normal{Float64}(μ=6.0, σ=1.0), Normal{Float64}(μ=7.0, σ=1.0), Normal{Float64}(μ=8.0, σ=1.0), Normal{Float64}(μ=9.0, σ=1.0), Normal{Float64}(μ=10.0, σ=1.0)]


NamedTupleDist((S = Uniform{Float64}(a=0.0, b=1.0), B = Uniform{Float64}(a=0.0, b=1.0), res = Product{Continuous, Uniform{Float64}, Vector{Uniform{Float64}}}(v=Uniform{Float64}[Uniform{Float64}(a=0.0, b=10.0)]), bias = DiagNormal(
dim: 10
μ: [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]
Σ: [1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 1.0]
)
))

In [42]:
maximum(Vector([1,2,3]))


3

In [39]:
typeof(truncated(Normal(0,1),0,Inf))

Truncated{Normal{Float64}, Continuous, Float64, Float64, Float64}

In [22]:
mode()

LoadError: UndefVarError: `mode` not defined

In [46]:

samples = bat_read("output/test_fit/mcmc_files/results.h5")
bat_report(samples.result)
BAT.bat_marginalmode(samples.result)

[ Info: Using input algorithm BATHDF5IO()


(result = (S = 2.4999999999999997e-27, B = 0.0005, res = [1.3, 1.2999999999999998, 1.125, 1.05, 1.125, 1.45, 1.5, 1.1, 1.45, 1.15, 1.25, 1.7000000000000002, 1.125], bias = [0.25, -0.3, 0.10000000000000003, -0.25, 0.10000000000000003, -0.25, -0.09999999999999998, 0.25, -0.25, -0.25, 0.10000000000000003, -0.75, -0.25]), optargs = (algorithm = BAT.BinnedModeEstimator{FreedmanDiaconisBinning}
  binning: FreedmanDiaconisBinning FreedmanDiaconisBinning()
, context = BATContext{Float64}(Random123.Philox4x{UInt64, 10}(0x3278e2e056c47afb, 0xb54851f267af6714, 0xbbb56e4dd588ee20, 0x9d9a5607dc282172, 0xfd29864eb96b8ec2, 0x9f71ce3f4a6f42f4, 0x0000000000000000, 0x0000000000000000, 0x0000000000000000, 0x0000000000000000, 0), HeterogeneousComputing.CPUnit(), BAT._NoADSelected())))

In [47]:
bat_report(samples.result)


### Sampling result

  * Total number of samples: 88569
  * Total weight of samples: 399987
  * Effective sample size: between 1697 and 2861

#### Marginals

| Parameter | Mean        | Std. dev.   | Gobal mode  | Marg. mode |       Cred. interval       | Histogram                                                  |
|:--------- |:----------- |:----------- |:----------- |:---------- |:--------------------------:|:---------------------------------------------------------- |
| S         | 3.59757e-27 | 3.52491e-27 | 6.95432e-29 | 2.5e-27    | 7.64607e-32 .. 4.11739e-27 | 7.65e-32[█▇▆▄▃▂▂▁▁▁▁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀[3.24e-26 |
| B         | 0.000555594 | 0.000144066 | 0.000611156 | 0.0005     | 0.000375949 .. 0.000666945 | 0.000135[⠀⠀⠀⠀⠀⠀▁▁▂▃▄▅▆▇███████▇▇▆▅▅▄▄▃▂▂▂▁▁▁⠀⠀⠀⠀⠀[0.000996 |
| res[1]    | 1.38796     | 0.113377    | 1.42437     | 1.3        |     1.27984 .. 1.50672     | ⠀⠀⠀⠀0.95[⠀⠀⠀⠀⠀⠀⠀⠀⠀▁▁▂▃▄▄▅▆▇████▇▆▅▄▃▂▂▁▁⠀⠀⠀⠀⠀⠀⠀⠀⠀[1.83⠀⠀⠀⠀ |
| res[2]    | 1.32814     | 0.0803328   | 1.39008     | 1.3        |     1.2433 .. 1.40487      | ⠀⠀⠀⠀1.01[⠀⠀⠀⠀⠀⠀⠀⠀⠀▁▁▂▃▄▅▆▇█████▇▆▅▄▃▂▁▁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀[1.67⠀⠀⠀⠀ |
| res[3]    | 1.10332     | 0.0285916   | 1.1163      | 1.125      |     1.07636 .. 1.1331      | ⠀⠀⠀0.982[⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀▁▁▂▃▄▅▆▇██████▆▅▄▃▂▁▁⠀⠀⠀⠀⠀⠀⠀⠀⠀[1.22⠀⠀⠀⠀ |
| res[4]    | 1.01439     | 0.0332445   | 1.02201     | 1.05       |    0.979927 .. 1.04585     | ⠀⠀⠀0.859[⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀▁▁▂▃▄▆▇▇████▇▆▅▄▃▂▁▁⠀⠀⠀⠀⠀⠀⠀⠀[1.14⠀⠀⠀⠀ |
| res[5]    | 1.10373     | 0.0286413   | 1.0965      | 1.125      |     1.07619 .. 1.13363     | ⠀⠀⠀0.988[⠀⠀⠀⠀⠀⠀⠀⠀⠀▁▁▂▃▄▅▇▇████▇▆▅▃▂▂▁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀[1.24⠀⠀⠀⠀ |
| res[6]    | 1.45421     | 0.0361551   | 1.44725     | 1.45       |     1.41627 .. 1.48875     | ⠀⠀⠀⠀1.28[⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀▁▁▂▃▄▅▇▇████▇▆▅▃▂▂▁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀[1.61⠀⠀⠀⠀ |
| res[7]    | 1.56709     | 0.10706     | 1.56474     | 1.5        |     1.46414 .. 1.67985     | ⠀⠀⠀⠀1.16[⠀⠀⠀⠀⠀⠀⠀⠀⠀▁▁▂▃▄▅▆▇██████▇▆▅▄▃▂▁▁▁⠀⠀⠀⠀⠀⠀⠀⠀[1.96⠀⠀⠀⠀ |
| res[8]    | 1.09346     | 0.075731    | 1.09706     | 1.1        |     1.01423 .. 1.16526     | ⠀⠀⠀0.808[⠀⠀⠀⠀⠀⠀⠀⠀⠀▁▂▂▃▄▅▆▇█████▇▆▅▄▃▂▁▁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀[1.4⠀⠀⠀⠀⠀ |
| res[9]    | 1.4545      | 0.0358001   | 1.45966     | 1.45       |     1.42033 .. 1.49207     | ⠀⠀⠀⠀1.31[⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀▁▂▂▃▄▅▆▇█████▇▆▅▄▃▂▁▁⠀⠀⠀⠀⠀⠀⠀⠀⠀[1.59⠀⠀⠀⠀ |
| res[10]   | 1.1087      | 0.0430818   | 1.1329      | 1.15       |     1.06348 .. 1.15001     | ⠀⠀⠀0.941[⠀⠀⠀⠀⠀⠀⠀⠀⠀▁▁▂▃▄▅▆▇█████▇▆▅▄▃▂▂▁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀[1.28⠀⠀⠀⠀ |
| res[11]   | 1.21247     | 0.0384447   | 1.22769     | 1.25       |     1.17191 .. 1.24887     | ⠀⠀⠀⠀1.06[⠀⠀⠀⠀⠀⠀⠀⠀⠀▁▁▂▃▄▅▆▇█████▇▆▄▃▂▂▁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀[1.38⠀⠀⠀⠀ |
| res[12]   | 1.64735     | 0.11344     | 1.58739     | 1.7        |     1.52969 .. 1.75567     | ⠀⠀⠀⠀1.18[⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀▁▁▂▃▄▅▆▇████▇▇▆▅▄▃▂▁▁⠀⠀⠀⠀⠀⠀⠀⠀⠀[2.1⠀⠀⠀⠀⠀ |
| res[13]   | 1.12608     | 0.0288283   | 1.11639     | 1.125      |     1.0988 .. 1.15615      | ⠀⠀⠀0.999[⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀▁▁▂▃▄▅▆▇████▇▆▅▃▂▁▁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀[1.26⠀⠀⠀⠀ |
| bias[1]   | 0.0657536   | 0.177157    | 0.0584621   | 0.25       |   -0.122608 .. 0.236234    | ⠀⠀-0.707[⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀▁▁▂▃▄▅▆▇████▇▇▆▄▄▂▂▁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀[0.82⠀⠀⠀⠀ |
| bias[2]   | -0.337863   | 0.0759473   | -0.354525   | -0.3       |   -0.409875 .. -0.257869   | ⠀⠀-0.663[⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀▁▁▂▃▄▅▆▇█████▇▆▅▄▂▂▁⠀⠀⠀⠀⠀⠀⠀⠀⠀[-0.0498⠀ |
| bias[3]   | 0.0240791   | 0.101726    | 0.0666474   | 0.1        |   -0.0723111 .. 0.131667   | ⠀⠀⠀-0.41[⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀▁▁▂▃▄▅▆▇█████▇▆▄▄▃▂▁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀[0.445⠀⠀⠀ |
| bias[4]   | -0.174198   | 0.214969    | -0.0616825  | -0.25      |   -0.379458 .. 0.0524542   | ⠀⠀⠀⠀⠀⠀-1[⠀⠀⠀⠀⠀⠀⠀⠀⠀▁▂▂▃▄▆▇██████▇▆▅▄▃▂▁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀[0.734⠀⠀⠀ |
| bias[5]   | 0.0277839   | 0.101532    | 0.0255375   | 0.1        |   -0.0776055 .. 0.123962   | ⠀⠀-0.357[⠀⠀⠀⠀⠀⠀⠀⠀⠀▁▁▂▃▃▄▆▇▇████▇▆▅▄▄▃▂▁▁⠀⠀⠀⠀⠀⠀⠀⠀⠀[0.411⠀⠀⠀ |
| bias[6]   | -0.0712613  | 0.14093     | 0.0412513   | -0.25      |   -0.205489 .. 0.0783964   | ⠀⠀-0.707[⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀▁▁▂▃▄▅▆▇████▇▆▅▄▃▂▁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀[0.521⠀⠀⠀ |
| bias[7]   | -0.0692296  | 0.105023    | -0.0471331  | -0.1       |   -0.169421 .. 0.0408439   | ⠀⠀-0.488[⠀⠀⠀⠀⠀⠀⠀⠀⠀▁▁▂▃▄▅▆▇████▇▆▅▄▃▂▁▁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀[0.389⠀⠀⠀ |
| bias[8]   | 0.0292449   | 0.204292    | 0.0374971   | 0.25       |   -0.163426 .. 0.241551    | ⠀⠀-0.701[⠀⠀⠀⠀⠀⠀⠀▁▁▂▃▄▅▆▇█████▇▆▄▃▂▁▁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀[0.985⠀⠀⠀ |
| bias[9]   | -0.0765742  | 0.143085    | -0.109422   | -0.25      |   -0.212123 .. 0.0739828   | ⠀⠀-0.672[⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀▁▁▂▂▃▄▅▆▇████▇▆▅▅▃▂▂▁▁⠀⠀⠀⠀⠀⠀⠀⠀[0.454⠀⠀⠀ |
| bias[10]  | -0.00281331 | 0.168849    | 0.0508186   | -0.25      |   -0.179797 .. 0.157766    | ⠀⠀-0.734[⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀▁▁▂▃▄▅▆▇████▇▆▅▄▃▂▁▁⠀⠀⠀⠀⠀⠀⠀⠀⠀[0.657⠀⠀⠀ |
| bias[11]  | 0.0571325   | 0.106468    | 0.0600344   | 0.1        |   -0.0546105 .. 0.157602   | ⠀⠀-0.343[⠀⠀⠀⠀⠀⠀⠀⠀⠀▁▂▂▃▄▅▆▇█████▇▆▆▄▄▃▂▁▁⠀⠀⠀⠀⠀⠀⠀⠀⠀[0.465⠀⠀⠀ |
| bias[12]  | -0.596356   | 0.14692     | -0.502893   | -0.75      |   -0.752675 .. -0.456732   | ⠀⠀⠀-1.19[⠀⠀⠀⠀⠀⠀⠀⠀⠀▁▁▂▃▄▅▆▇████▇▆▅▄▃▂▁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀[0.0788⠀⠀ |
| bias[13]  | -0.110837   | 0.132684    | 0.0165108   | -0.25      |   -0.238074 .. 0.027072    | ⠀⠀-0.682[⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀▁▁▂▃▄▅▅▇▇████▇▆▄▃▂▂▁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀[0.433⠀⠀⠀ |
